<a href="https://colab.research.google.com/github/hammad93/hurricane-tts/blob/main/hurricane_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
Please make sure to run all tests.

In [1]:
%cd ~/
!git clone https://github.com/hammad93/hurricane-tts.git

/root
Cloning into 'hurricane-tts'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 268 (delta 33), reused 0 (delta 0), pack-reused 195
Receiving objects: 100% (268/268), 1.26 MiB | 13.47 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [2]:
!pip install -r hurricane-tts/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
import os
if not os.getenv("AZURE_OPENAI_API_KEY") :
  from google.colab import userdata
  os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
if not os.getenv("AZURE_REDIS_KEY") :
  os.environ["AZURE_REDIS_KEY"] = userdata.get('AZURE_REDIS_KEY')

In [4]:
%cd ~/hurricane-tts
!python ~/hurricane-tts/test.py

/root/hurricane-tts
.Here are the constructed messages: [{'role': 'system', 'content': 'You are an AI assistant that helps people find information.'}, {'role': 'user', 'content': 'test'}]
..
----------------------------------------------------------------------
Ran 3 tests in 7.272s

OK


In [5]:
import prompts
import utils
import db
# we generate prompts by ingesting live hurricane data and supported languages
storm_data = utils.transform_storm_data()
prompt_data = prompts.generate_prompts()

# metadata related to multilingual generation
supported_langs = prompts.unique_lang_list()
supported_langs_df = utils.tts_langs()

In [6]:
# example
print(prompt_data['storms'][0])

Provided are the tropical storm records with geographic coordinates and wind speed in knots.
Please respond with at least 2 of the supported languages by utilizing language geography and the storm's latitude and longitude.
The response must be a Python list meant to be input as a string into eval(). Please make sure the languages are in the supported list.
As an example, ["Chinese, Mandarin", "Haitian Creole", "Indonesian"]
[{'type': 'history', 'lat': 13.5, 'lon': 87.0, 'time': '2023-11-14 18:00:00', 'wind_speed': 20}, {'type': 'history', 'lat': 14.1, 'lon': 86.9, 'time': '2023-11-15 00:00:00', 'wind_speed': 20}, {'type': 'history', 'lat': 14.8, 'lon': 86.8, 'time': '2023-11-15 06:00:00', 'wind_speed': 20}, {'type': 'history', 'lat': 15.4, 'lon': 86.7, 'time': '2023-11-15 12:00:00', 'wind_speed': 20}, {'type': 'history', 'lat': 16.3, 'lon': 86.8, 'time': '2023-11-15 18:00:00', 'wind_speed': 25}, {'type': 'history', 'lat': 17.3, 'lon': 87.2, 'time': '2023-11-16 00:00:00', 'wind_speed': 

# Language Geography inference from tropical storm geographical coorindates
We're trying to answer the question, "Which languages do we produce speech for to report on this tropical storm?" This pipeline will produce an English output but also utilize massively multilingual capabilities such that emergency notices are also in local languages

In [8]:
storm_langs = {} # keys are storm id's and values are the languages
storm_chats = {} # stores chat histories for storms
for index, storm_prompt in enumerate(prompt_data['storms']) :
  storm_id = list(storm_data.keys())[index]
  # Get the languages to generate the report for this storm
  retries = 20 # sometimes it fails, so we retry it
  while retries > 0 :
    response = prompts.chat(system=prompt_data['system'], message=storm_prompt)
    result = utils.llm_response_transform(
        resp=response,
        supported_langs=supported_langs)
    if result :
      print(f'Successfully parsed {response}')
      break
    else :
      retries = retries - 1
      print(f"Failed. Retries left: {retries}")
  if retries < 1 :
    raise Exception("Couldn't produce a correct output from LLM.")

  # store results
  storm_langs[storm_id] = {'names': ['English'] + result}
  storm_chats[storm_id] = {
      'history' : [{'role': 'system', 'content': prompt_data['system']},
                   {'role': 'user', 'content': storm_prompt},
                   {'role': 'assistant', 'content': response}]
  }

Here are the constructed messages: [{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nThis is the list of supported languages,\n{'KabiyÃ¨', 'PaumarÃ\\xad', 'Gorontalo', 'Dhimal', 'Gwichâ\\x80\\x99in', 'Sangir', 'Tepehua, Huehuetla', 'Totonac, Highland', 'Mixtec, Northern Tlaxiaco', 'Chinantec, OzumacÃ\\xadn', 'Tsafiki', 'Chinantec, Lalana', 'Nahuatl, Eastern Huasteca', 'Delo', 'Tzâ\\x80\\x99utujil', 'KimrÃ©', 'Kilivila', 'Mixtec, Southern Puebla', 'Azerbaijani, South', 'Hebrew', 'Mwan', 'Sasak', 'Purepecha', 'Gaddi', 'Lole', 'Kuman', 'Khmer', 'Sampang', 'Karaboro, Eastern', 'Izii', 'Bamanankan', 'Nagamese', 'Ayta, Mag-Indi', 'Mende', 'Popoluca, Highland', 'SÃ©noufo, Supyire', 'Torres Strait Creole', 'Mixtec, Magdalena PeÃ±asco', 'Churahi', 'Sharanahua', 'Icelandic', 'Tunebo, Central', 'Migabac', 'Dawro', 'Chorote, Iyojwaâ\\x80\\x99ja', 'Nzema', 'Sylheti', 'Kumyk', 'Odia', 'Chinese, Hakka', 'Ngangam', 'Bauzi', 'Kisar', 'Aukan', 'Pui

# Tropical Storm Report
This code will create the report in the language specified based on the tropical storm.

In [10]:
with open('prompts/report-prompt.txt', 'r') as file:
  report_prompt = file.read()
for storm in storm_langs:
  print(storm)
  reports = []
  for lang in storm_langs[storm]['names']:
    print(storm_chats[storm]['history'])
    # Construct the prompt
    message = report_prompt.format(lang=lang, id=storm[2:-4])
    print(message)
    # Send to LLM
    response = prompts.chat(
        message = message, history = storm_chats[storm]['history'])
    print(response)
    # store data
    reports.append(response)
  storm_langs[storm]['reports'] = reports

IO72023
[{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nThis is the list of supported languages,\n{'KabiyÃ¨', 'PaumarÃ\\xad', 'Gorontalo', 'Dhimal', 'Gwichâ\\x80\\x99in', 'Sangir', 'Tepehua, Huehuetla', 'Totonac, Highland', 'Mixtec, Northern Tlaxiaco', 'Chinantec, OzumacÃ\\xadn', 'Tsafiki', 'Chinantec, Lalana', 'Nahuatl, Eastern Huasteca', 'Delo', 'Tzâ\\x80\\x99utujil', 'KimrÃ©', 'Kilivila', 'Mixtec, Southern Puebla', 'Azerbaijani, South', 'Hebrew', 'Mwan', 'Sasak', 'Purepecha', 'Gaddi', 'Lole', 'Kuman', 'Khmer', 'Sampang', 'Karaboro, Eastern', 'Izii', 'Bamanankan', 'Nagamese', 'Ayta, Mag-Indi', 'Mende', 'Popoluca, Highland', 'SÃ©noufo, Supyire', 'Torres Strait Creole', 'Mixtec, Magdalena PeÃ±asco', 'Churahi', 'Sharanahua', 'Icelandic', 'Tunebo, Central', 'Migabac', 'Dawro', 'Chorote, Iyojwaâ\\x80\\x99ja', 'Nzema', 'Sylheti', 'Kumyk', 'Odia', 'Chinese, Hakka', 'Ngangam', 'Bauzi', 'Kisar', 'Aukan', 'Puinave', 'Ganda', 'Mambila, N

In [11]:
storm_langs

{'IO72023': {'names': ['English', 'Bengali', 'Hindi'],
  'reports': ["Welcome to this message about storm number 7. This storm is expected to bring strong winds and heavy rainfall to the affected areas. Please stay tuned to local weather updates and follow all evacuation orders from officials. It's important to remember that storms like this can cause significant damage and loss of life. We also encourage you to learn from past storms and how their impact was mitigated. Together, we can prepare and stay safe during this storm.",
   'পিছলের কিছু তুফান সম্পর্কে জানা প্রয়োজন যেগুলি বাংলাদেশ ও পূর্ব ভারতের উত্তরাঞ্চলে আক্রমণ করে। অনেকগুলি লোক তার পরিসরের সাথে সম্পর্কিত আবার তার পূর্বের তুফান জ্ঞান দিয়ে নিজেদের তাদের জীবন উন্নয়ন করতে পারে। তুফান নম্বর 7 জন্য নির্দেশিকা প্রয়োজনীয় সমস্ত তথ্য এবং সচেতনতা সরবরাহ করবে যাতে এমন একটি প্রকল্প হয় যা স্থানীয় জনগণের বুদ্ধিমান বোধ করে। (60 সেকেন্ডের আউডিও)',
   'उस समय जब एक तूफ़ान स्थानीय समुद्र के ऊपर तेज़तर बनता है, तो यह आपके समुद्र तट के लो

The following are from the TTS implementation
https://github.com/facebookresearch/fairseq/blob/main/examples/mms/tts/tutorial/MMS_TTS_Inference_Colab.ipynb


In [12]:
%cd ~/
!git clone https://github.com/jaywalnut310/vits.git
%cd vits/

!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install scipy
!pip install "numpy<1.24"
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode==1.1.1

%cd monotonic_align/
%mkdir monotonic_align
!python3 setup.py build_ext --inplace
%cd ../
%pwd

/root
Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Receiving objects: 100% (81/81), 3.33 MiB | 16.65 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/root/vits
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.5
    Uninstalling Cython-3.0.5:
      Successfully uninstalled Cython-3.0.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.1 MB/s eta 0:00:00
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201367 sha256=103e71146bb977cd52d2411bb25ddebbbd519b08e818e4aea81ee4c37c2bc150
  Stored in directory: /root/.cache/pip/wheels/bf/b7/85/2f8044306ccec014930aea23ad4852fca9e2584e21c6972bc6
Successfully built librosa
  Attempting uninstall: librosa
  

'/root/vits'

In [13]:
# we have the data we need and the utils class is also used in the next code
import sys
del sys.modules["utils"]

In [14]:
%cd ~/hurricane-tts/
import fair_mms_tts
%cd ~/vits
import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn

/root/hurricane-tts
/root/vits


In [15]:
# match language names to iso code
for storm in storm_langs :
  langs = storm_langs[storm]['names']
  print(langs)
  codes = [supported_langs_df[
      supported_langs_df['Language Name'] == name
  ]['ISO 693-3'].iloc[0] for name in langs]
  print(codes)
  storm_langs[storm]['codes'] = codes

['English', 'Bengali', 'Hindi']
['eng', 'ben', 'hin']
['English', 'Tok Pisin', 'Spanish']
['eng', 'tpi', 'spa']
['English', 'Indonesian', 'Tagalog']
['eng', 'ind', 'tgl']
['English', 'Tagalog', 'Indonesian']
['eng', 'tgl', 'ind']
['English', 'Marshallese', 'Samoan']
['eng', 'mah', 'smo']
['English', 'Spanish', 'Haitian Creole', 'Indonesian']
['eng', 'spa', 'hat', 'ind']
['English', 'Bengali', 'Hindi', 'Burmese']
['eng', 'ben', 'hin', 'mya']


In [16]:
import soundfile as sf
def generate_audio(report, lang_code, out):
  '''
  Generates the audio given the report and lang_code
  '''
  ckpt_dir = fair_mms_tts.download(lang_code)
  import torch
  if torch.cuda.is_available():
      device = torch.device("cuda")
  else:
      device = torch.device("cpu")
  print(device)
  print(f"Run inference with {device}")
  vocab_file = f"{ckpt_dir}/vocab.txt"
  config_file = f"{ckpt_dir}/config.json"
  assert os.path.isfile(config_file), f"{config_file} doesn't exist"
  hps = utils.get_hparams_from_file(config_file)

  text_mapper = fair_mms_tts.TextMapper(vocab_file)
  net_g = SynthesizerTrn(
      len(text_mapper.symbols),
      hps.data.filter_length // 2 + 1,
      hps.train.segment_size // hps.data.hop_length,
      **hps.model)
  net_g.to(device)
  _ = net_g.eval()

  g_pth = f"{ckpt_dir}/G_100000.pth"
  print(f"load {g_pth}")

  _ = utils.load_checkpoint(g_pth, net_g, None)
  txt = fair_mms_tts.preprocess_text(report, text_mapper, hps, lang=lang_code)
  stn_tst = text_mapper.get_text(txt, hps)

  with torch.no_grad():
      x_tst = stn_tst.unsqueeze(0).to(device)
      x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
      hyp = net_g.infer(
          x_tst, x_tst_lengths, noise_scale=.667,
          noise_scale_w=0.8, length_scale=1.0
      )[0][0,0].cpu().float().numpy()

  print(f"Generated audio")
  sf.write(out, hyp, hps.data.sampling_rate)

In [19]:
# generate audio
import time
time_fname = int(time.time())
outputs = []
outputs_metadata = {}
for storm in storm_langs :
  print(storm)
  current = storm_langs[storm]
  for index, code in enumerate(current['codes']):
    path = f"/root/{storm}_{time_fname}_{current['names'][index]}.wav"
    prompt = current['reports'][index]
    outputs.append(path)
    outputs_metadata[path] = {
        'prompt': prompt,
        'lang': code
    }
    generate_audio(report = prompt,
                   lang_code = code,
                   out = path)

Download model for language: eng
Model checkpoints in ./eng: ['G_100000.pth', 'vocab.txt', 'config.json']
cuda
Run inference with cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


load ./eng/G_100000.pth
eng
text after filtering OOV: welcome to this message about storm number  this storm is expected to bring strong winds and heavy rainfall to the affected areas please stay tuned to local weather updates and follow all evacuation orders from officials it's important to remember that storms like this can cause significant damage and loss of life we also encourage you to learn from past storms and how their impact was mitigated together we can prepare and stay safe during this storm
Generated audio
Download model for language: ben
Model checkpoints in ./ben: ['G_100000.pth', 'vocab.txt', 'config.json']
cuda
Run inference with cuda
load ./ben/G_100000.pth
ben
text after filtering OOV: পিছলের কিছু তুফান সম্পর্কে জানা প্রয়োজন যেগুলি বাংলাদেশ ও পূর্ব ভারতের উত্তরাঞ্চলে আক্রমণ করে অনেকগুলি লোক তার পরিসরের সাথে সম্পর্কিত আবার তার পূর্বের তুফান জ্ঞান দিয়ে নিজেদের তাদের জীবন উন্নয়ন করতে পারে তুফান নম্বর 7 জন্য নির্দেশিকা প্রয়োজনীয় সমস্ত তথ্য এবং সচেতনতা সরবরাহ করবে যা

In [20]:
import db
import json
print(outputs)
db.upload_latest_audios(outputs, outputs_metadata)

['/root/IO72023_1700206105_English.wav', '/root/IO72023_1700206105_Bengali.wav', '/root/IO72023_1700206105_Hindi.wav', '/root/SH22023_1700206105_English.wav', '/root/SH22023_1700206105_Tok Pisin.wav', '/root/SH22023_1700206105_Spanish.wav', '/root/WP172023_1700206105_English.wav', '/root/WP172023_1700206105_Indonesian.wav', '/root/WP172023_1700206105_Tagalog.wav', '/root/WP982023_1700206105_English.wav', '/root/WP982023_1700206105_Tagalog.wav', '/root/WP982023_1700206105_Indonesian.wav', '/root/WP972023_1700206105_English.wav', '/root/WP972023_1700206105_Marshallese.wav', '/root/WP972023_1700206105_Samoan.wav', '/root/AL222023_1700206105_English.wav', '/root/AL222023_1700206105_Spanish.wav', '/root/AL222023_1700206105_Haitian Creole.wav', '/root/AL222023_1700206105_Indonesian.wav', '/root/IO942023_1700206105_English.wav', '/root/IO942023_1700206105_Bengali.wav', '/root/IO942023_1700206105_Hindi.wav', '/root/IO942023_1700206105_Burmese.wav']


In [ ]:
!git -C /root/hurricane-tts/ pull